In [1]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
import os
from dotenv import load_dotenv
import psycopg2

In [2]:
load_dotenv()

# Access environment variables
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")

In [3]:
df = pd.read_csv('marks.csv')
df.head()

,student_class_id,marks
0,1,10
1,2,20
2,3,32
3,4,40
4,9,50


In [4]:
out_of = 100

In [5]:
df['marks'] = (df['marks'] / out_of) * 100
df.head()

,student_class_id,marks
0,1,10.0
1,2,20.0
2,3,32.0
3,4,40.0
4,9,50.0


In [6]:
X = df.drop("student_class_id", axis=1)

gmm = GaussianMixture(n_components=5, random_state=0)
df['tier'] = gmm.fit_predict(X)
df.head()

,student_class_id,marks,tier
0,1,10.0,3
1,2,20.0,3
2,3,32.0,0
3,4,40.0,0
4,9,50.0,4


In [7]:
tier_df = df[["student_class_id", "tier"]]

# Show DataFrame
tier_df

,student_class_id,tier
0,1,3
1,2,3
2,3,0
3,4,0
4,9,4
5,121,4
6,122,1
7,123,1
8,124,2
9,125,2


In [12]:
import psycopg2
from datetime import datetime

conn = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password
)



def update_institute_class_students(conn, update_data):
    class_id = 1
    cur = conn.cursor()

    for index, row in update_data.iterrows():
        student_id = int(row['student_class_id']) 
        tier = int(row['tier'])  

        update_query = """
        UPDATE institute_class_students
        SET tier = %s, tier_last_updated_at = %s
        WHERE student_id = %s and class_id = %s
        """
        
        current_time = datetime.now()
        # Execute the update query with the parameters
        cur.execute(update_query, (tier, current_time, student_id, class_id))

    # Commit the changes to the database
    conn.commit()
    cur.close()

update_institute_class_students(conn, tier_df)

In [ ]:
from datetime import datetime

def insert_and_update_institute_class_tier_students(conn, insert_data,class_id):
    cur = conn.cursor()

    for index, row in insert_data.iterrows():
        student_id = int(row['student_class_id']) 
        tier = int(row['tier'])
        marks = float(row['marks'])  
        out_of = 100  

        select_query = """
            SELECT class_id, institute_id 
            FROM institute_class_students 
            WHERE student_id = %s and class_id = %s
        """
        cur.execute(select_query, (student_id, class_id))
        result = cur.fetchone()

        if result is not None:
            institute_id = result[1]

            print(class_id, institute_id)

            select_student_class_id_query = """
                SELECT id
                FROM institute_class_students
                WHERE student_id = %s and class_id = %s
            """
            cur.execute(select_student_class_id_query, (student_id, class_id))
            result_1 = cur.fetchone()

            print(result_1)

            if result_1 is not None:
                
                studet_class_id = result_1[0]

                insert_query = """
                    INSERT INTO institute_class_tier_students (student_class_id, class_id, institute_id, tier, marks, marks_out_of, upload_date)
                    VALUES (%s, %s, %s, %s, %s, %s, %s)  -- Add a placeholder for each parameter
                """
                cur.execute(insert_query, (student_id, class_id, institute_id, tier, marks, out_of, datetime.now()))

                # Update the student's tier in the institute_class_students table.
                update_query = """
                    UPDATE institute_class_students
                    SET tier = %s, tier_last_updated_at = %s
                    WHERE id = %s
                """
                cur.execute(update_query, (tier, datetime.now(), student_id))

        else:
            continue
        
    conn.commit()
    cur.close()

# Example call
insert_and_update_institute_class_tier_students(conn, df, 1)


1 1
1 1
